# World Airport Imagery Retrieval


In [1]:
%config IPCompleter.greedy=True
import folium
import os
from zipfile import ZipFile
import requests
from six.moves.urllib.parse import urljoin
from tqdm import tqdm
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
from pyproj import Proj, CRS,transform
import gdal
from datetime import datetime
import pygc
from io import StringIO
import shutil
from osgeo import gdal, gdal_array
from PIL import Image
import json

In [2]:
missing_values = [r"\N"]
df0 = pd.read_csv(r'airport-codes_csv.csv',na_values = missing_values)
list1,list2,a,b = [],[],[],[]
for itema,itemb in zip(df0["iata_code"],df0['iso_region']):
    list1.append(itema)
    list2.append(itemb)
    
for items in df0['coordinates']:
    a.append(items.split(',')[0])
    b.append(items.split(',')[1])

In [3]:
iatalist,iatabatch= [],[]
batch_ranges = []
batch_size = 2
batch_number = 2
missing_values = [r"\N"]
#df = pd.read_excel(r'190812_CompanyRelations.xlsx', sheet_name = 'Airport Data',na_values = missing_values)
df = pd.read_csv(r'airportDB.csv')
df_all = df
#.sort_values(by = ['2018 Passengers'], ascending = False)
#t = np.invert(df['2018 Passengers'].isnull())
#df = df[t]
#df = df.sort_values(by = ["2018 Passengers"], ascending = False) 
allNames = []
for items,names in zip(df_all['IATA'],df_all['Name']):
    iatabatch.append(items)
    allNames.append(names)
    
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

iatabatch = Remove(iatabatch)
chunk_items = len(iatabatch) - 1

for i in range(chunk_items):
    batch_ranges.append(iatabatch[i*batch_size : (i+1)*batch_size])
    
batch_ranges

for i in range(batch_size):
    try:
        iatalist.append(batch_ranges[batch_number -1][i])
    except:
        continue
iatalist,allNames[:1000]
air = dict(zip(iatalist,allNames[:1000]))

In [4]:
iatalist

['DXB', 'LAX']

In [5]:
iata,iatalat,iatalon,notthere = [],[],[],[]
for key in iatalist:
    if key in list1:
        iata.append(key)
        #have changed codes --> list1 & codes[key].lat/lon --> a/b[list1.index(key)]
        iatalon.append(a[list1.index(key)])
        iatalat.append(b[list1.index(key)])
    else : 
        notthere.append(key)
    

#print(iata[0],iatalat[0],iatalon[0])


len(iatalat)

#printing the entries that we have in our excel but not in our database(significantly improved results over the .dat file previously supplied)
notthere

[]

Using Python graphic library, we try to virtually construct a geo-boundary translated from our required distance (typically around 10-20kms). 

In [6]:
def latLonBoxByWandH(lat,lon,ew_width,ns_height):
    lats, lons = [], []
    #distance in m, az (in deg), lat (in deg), long (in deg)

    res = pygc.great_circle(distance=ew_width/2, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']

    res = pygc.great_circle(distance=ns_height/2, azimuth=180, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=270, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ns_height, azimuth=0, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)
    
    return {'lats':lats,'lons':lons}


#test the function
#lengths in m
ew_width = 10000
ns_height = 10000
loc_lat,loc_lon = [], []
#loc_lat = [51.4775, 51.148056,1.359167]
#loc_lon = [-0.461389, -0.190278, 103.989444]
for i in range(0,len(iata)):
    loc_lat.append(iatalat[i])
    loc_lon.append(iatalon[i])
    #print(iata[i])

#loc_lon,loc_lat
loc_lat = np.asfarray(loc_lat,float)
loc_lon = np.asfarray(loc_lon,float)
len(loc_lat),len(loc_lon)

(2, 2)

Once we construct the spatial boundary, we then try to project in on a basemap of the globe and use Folium to try and do a sanity check if our geo-boundaries actually fall under the area of our concern, in this case airports.

In [7]:
polygons,footprint =[], []

for lat, lon in zip(loc_lat,loc_lon):

    box = latLonBoxByWandH(lat,lon,ew_width,ns_height)
    
    polygon_geom = Polygon(zip(box['lons'], box['lats']))
    footprint.append(polygon_geom)
    crs = CRS('epsg:4326')
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    
    polygons.append(polygon)


Proj("epsg:4326")

m = folium.Map([loc_lat[0],loc_lon[0]], zoom_start=12,tiles = "https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png",
attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>')
for polygon in polygons:
    folium.GeoJson(polygon).add_to(m)
    
m

m.save('500_formetaArticle.html')

In [8]:
user = 'demi12395' 
password = 'Sutd1234' 

api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
start = '20200412'
end = '20201215'
cloudperc = 50

In [9]:
def apiQuery(iatalist,geometry,dateStart,dateEnd,cloud):
    query = api.query(geometry,
                 date = (str(dateStart),str(dateEnd)),
                 platformname = 'Sentinel-2',
                 processinglevel = 'Level-2A',
                area_relation = ('Contains'),
                 cloudcoverpercentage = (0,cloud))
    return query
    #print(ia, len(query))

In [10]:
apiq = []
for ia,boundary in tqdm(zip(iatalist,footprint),total=len(iatalist)):
    queried = apiQuery(ia,boundary,start,end,cloudperc)
    while len(queried) == 0:
        queried = apiQuery(ia,boundary,start,end,cloudperc+10)
    apiq.append(queried)
    print(ia,len(queried))
   

 50%|██████████████████████████████████████████                                          | 1/2 [00:20<00:20, 20.19s/it]

DXB 48


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:36<00:00, 18.41s/it]

LAX 39


In [11]:
products_list, products_list_sorted,images,titlelist,bestlist,datestamp,cloudiness =[],[], [], [], [] , [],[]
for products in apiq:
        products_list.append(api.to_geodataframe(products))
        
#Sorting the list of products within our array of locations for minimum cloudcover
for products in products_list:
    products_list_sorted.append(products.sort_values(['cloudcoverpercentage'],ascending = [True]))
    

for i in range(0,len(products_list_sorted)):
        images.append(products_list_sorted[i].head(1))

for i in range(0,len(images)):
    titlelist.append(products_list_sorted[i].title[0])
    bestlist.append(products_list_sorted[i].uuid[0])
    d = products_list_sorted[i].beginposition[0].date().strftime("%Y%m%d")
    c = products_list_sorted[i].cloudcoverpercentage
    datestamp.append(d)
    cloudiness.append(c)

C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [12]:
#datestamp

In [13]:
#iatalist

In [14]:
prodlist = bestlist
#prodlist

In [15]:
def getImageURLs(prod_id,iata):
    urls,fNames = [],[]


    # connect to the api
    api_session = requests.Session()
    api_session.auth = ("demi12395", "Sutd1234")
    api_url = "https://scihub.copernicus.eu/apihub/odata/v1/"

    # product UUID you want to download a single band for
    for ids,ia in tqdm(zip(prod_id,iata),total=len(iata)):
        #print(ia)
        # parse the product name
        nodes = api_session.get(urljoin(api_url, "Products('%s')/Nodes?$format=json" % ids)).json()
        prod_name = nodes["d"]["results"][0]["Id"]
    #     print('prod_name {}'.format(prod_name))

        # parse the granule id
        granules = api_session.get(urljoin(api_url, "Products('%s')/Nodes('%s')/Nodes('GRANULE')/Nodes?$format=json" % (ids, prod_name))).json()
        granules["d"]["results"][0].keys()
        gran_id = granules["d"]["results"][0]["Id"]
    #     print('granule {}'.format(gran_id))



        # parse the band names
        bands = api_session.get(urljoin(api_url, "Products('%s')/Nodes('%s')/Nodes('GRANULE')/Nodes('%s')/Nodes('IMG_DATA')/Nodes?$format=json" % (ids, prod_name, gran_id))).json()
        band_id = bands["d"]["results"][0]["Id"]  # element 3 is band 4, element 0 band 1
    #     print('band {}'.format(band_id))


        # construct the final image url
        TCI_url = api_session.get(urljoin(api_url, "Products('%s')/Nodes('%s')/Nodes('GRANULE')/Nodes('%s')/Nodes('IMG_DATA')/Nodes('%s')/Nodes?$format=json" % (ids, prod_name, gran_id, band_id))).json()
        TCI_id = TCI_url["d"]["results"][5]["Id"]
    #     print('TCI_ID {}'.format(TCI_id))


        img_url = urljoin(api_url, "Products('%s')/Nodes('%s')/Nodes('GRANULE')/Nodes('%s')/Nodes('IMG_DATA')/Nodes('%s')/Nodes('%s')/$value" % (ids, prod_name, gran_id, band_id,TCI_id))
        urls.append(img_url)
        fNames.append(TCI_id)
    return urls,fNames

In [16]:
def getImages(fNames,urls,iata):
    for names,urls,ia in tqdm(zip(fNames,urls,iata),total = len(iata)):

        r = requests.get(urls,auth = (user,password))
        with open("Products_R/" + str(ia) + "_" + str(names),'wb') as f: 
            f.write(r.content) 
        f.close()

In [17]:
urlLinks,tileNames = getImageURLs(prodlist,iatalist)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

prod_name S2B_MSIL2A_20200910T064629_N0214_R020_T40RCN_20200910T092724.SAFE
granule L2A_T40RCN_A018350_20200910T065646
band R10m


 50%|██████████████████████████████████████████                                          | 1/2 [00:05<00:05,  5.34s/it]

TCI_ID T40RCN_20200910T064629_TCI_10m.jp2
prod_name S2A_MSIL2A_20201104T183541_N0214_R027_T11SLT_20201104T210809.SAFE
granule L2A_T11SLT_A028052_20201104T183645
band R10m


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.76s/it]

TCI_ID T11SLT_20201104T183541_TCI_10m.jp2


In [19]:
urlLinks[1]

"https://scihub.copernicus.eu/apihub/odata/v1/Products('6b70097e-0b03-4d15-b0ba-780423103205')/Nodes('S2A_MSIL2A_20201104T183541_N0214_R027_T11SLT_20201104T210809.SAFE')/Nodes('GRANULE')/Nodes('L2A_T11SLT_A028052_20201104T183645')/Nodes('IMG_DATA')/Nodes('R10m')/Nodes('T11SLT_20201104T183541_TCI_10m.jp2')/$value"

In [98]:
getImages(tileNames,urlLinks,iatalist)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [35:38<00:00, 21.38s/it]


In [131]:
path = "Products_R/"
sortedFileList = []
for ia in iatalist:
    for files in os.listdir(path):
        if files.endswith(".jp2") and files[:3] == ia:
            sortedFileList.append(files)
#sortedFileList

In [133]:
#sortedFileList

In [134]:
rankList = list(dataRanks.keys())

In [135]:
rankList.index('TMP')

1400

In [137]:
#iatalist

In [138]:
fileNames,file = [],[]
targetPath = "Products/"
for items,meta,ia in tqdm(zip(sortedFileList,dataArr,iatalist),total = len(iatalist)):
    openFile = path + str(items)
    saveFile = targetPath + str(rankList.index(str(ia)) + 2) + "_"  + str(items)[:-4] + ".tiff"
    #print(openFile)
    input_raster = gdal.Open(openFile)
    #print(saveFile)
    fileNames.append(saveFile)
    gdal.Warp(saveFile,openFile,srcSRS = str(meta.meta['crs']),dstSRS = 'epsg:4326',width = str(meta.meta['width']),height = str(meta.meta['height']))


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [22:53<00:00, 13.74s/it]


In [139]:
#fileNames

In [140]:
for items in fileNames:
    data = rio.open(items)
    #print(items)
    print(data.meta['crs'])
    data.close()

EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326


In [141]:
geoms = []
for i in range(0,len(datestamp)):
    geoms.append(polygons[i]['geometry'])

IndexError: list index out of range

In [142]:
len(geoms)

99

In [107]:
from rasterio.mask import mask
for names,g,date,ia in tqdm(zip(fileNames,geoms,datestamp,iatalist),total = len(fileNames)):
    with rasterio.open(names) as src:
            out_image, out_transform = mask(src, g, crop=True)
            out_meta = src.meta.copy()

    out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
    try:
        with rasterio.open("Source/S" + "_" +  ia + "_" + date  +".tiff", "w", **out_meta) as dest:
            dest.write(out_image)
        #os.remove(titlelist[k][i])
        src.close()
        dest.close()
    except:
        continue
        

  0%|                                                                                           | 0/99 [00:00<?, ?it/s]


ValueError: Input shapes do not overlap raster.